In [1]:
from utils import *
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv("data/M.csv") \
                .pipe(clean_data) 

In [3]:
df_despacho = df.pipe(return_despacho_movements)
df_hogar = df.pipe(return_hogar_movements)

In [4]:
df

,Fecha,Tipo,Categoria,Subcategoria,Beneficiario,Importe
0,2022-01-11,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.69
1,2022-02-11,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.69
2,2022-03-11,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.62
3,2022-04-11,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.69
4,2022-05-11,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.66
...,...,...,...,...,...,...
2187,2022-12-24,Gastos,Transferencia,NaN,METALICO,-5.75
2188,2022-12-25,Gastos,Transferencia,NaN,METALICO,-50.00
2189,2022-07-07,Gastos,Transferencia,NaN,IBERCAJA,-29.27
2190,2022-08-02,Gastos,Transferencia,NaN,IBERCAJA,-50.00


In [16]:
(df_despacho
     .pipe(pivot_by_category_totals, ["Tipo"])
#     .pipe(style_locale_es)
 )

,Tipo,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,-263.54,-490.46,-2310.61,-440.76,-283.38,-1430.33,-431.15,-593.03,-1782.86,-638.43,-655.18,-627.95,-828.97,-9947.68
1,Ingresos,1938.61,4446.89,8752.27,242.0,50.0,3903.71,3670.0,3413.3,4578.71,5334.69,3290.6,410.0,3335.9,40030.78
2,Total,1675.07,3956.43,6441.66,-198.76,-233.38,2473.38,3238.85,2820.27,2795.85,4696.26,2635.42,-217.95,2506.93,30083.1


In [17]:
(df_hogar
     .query("Fecha >= '2023-01-01'")
    .pipe(pivot_by_category_totals, ["Tipo"])
     .pipe(style_locale_es)
 )

,Tipo,Enero 2023,Media,Total
0,Gastos,"-7,99","-7,99","-7,99"
1,Total,"-7,99","-7,99","-7,99"


In [60]:
def treemap_data2(df):
    return (df
        .assign(MaxDate = lambda df: pd.to_datetime((df.Fecha.max() - datetime.timedelta(days=0)).date()),
                MinDate = lambda df: pd.to_datetime((df.Fecha.min() - datetime.timedelta(days=0)).date()))
        .query("Fecha.between(MinDate, MaxDate)")
        .pipe(pivot_by_category, ["Tipo", "Categoria", "Subcategoria", "Beneficiario"])
        # .filter(["Tipo", "Categoria", "Subcategoria", "Beneficiario", "Media", "Total"])
        .query("not (Tipo == 'Gastos' and Total > 0)")
        .assign(Media = lambda df: df.Media.abs(),
                Total = lambda df: df.Total.abs()
               )
     )

In [61]:
df_hogar.pipe(treemap_data2)

,Tipo,Categoria,Subcategoria,Beneficiario,Enero 2023,Diciembre 2022,Noviembre 2022,Octubre 2022,Septiembre 2022,Agosto 2022,Julio 2022,Junio 2022,Mayo 2022,Abril 2022,Marzo 2022,Febrero 2022,Enero 2022,Media,Total
0,Gastos,Cultura,Cine,RENOIR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.0,2.0,26.0
1,Gastos,Cultura,Conciertos,GIGSBERG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-281.0,0.0,0.0,0.0,21.62,281.0
2,Gastos,Cultura,Libros,AMAZON,0.0,0.0,0.0,0.0,-16.1,0.0,0.0,0.0,-22.75,0.0,-8.5,-8.07,-9.93,5.03,65.35
3,Gastos,Cultura,Libros,ESPASA CALPE,0.0,0.0,0.0,0.0,-23.95,0.0,0.0,0.0,0.0,0.0,-18.9,0.0,0.0,3.3,42.85
4,Gastos,Cultura,Libros,FABULA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-18.91,1.45,18.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Ingresos,Ingresos,Despacho,No Definido,0.0,1675.07,3956.43,6441.66,-198.76,-233.38,2473.38,3238.85,2820.27,2795.85,4696.26,2635.42,-217.95,2314.08,30083.1
347,Ingresos,Ingresos,Intereses Bancarios,OPENBANK,0.0,17.49,17.8,8.42,4.69,4.7,5.93,4.32,6.28,5.25,4.22,4.54,4.44,6.78,88.08
348,Ingresos,Ingresos,Intereses Bancarios,INGDIRECT,0.0,5.33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.41,5.33
349,Ingresos,Ingresos,Nómina,SANTANDER TECNOLOGIA,0.0,5878.24,2815.3,2815.71,2815.3,2815.3,2812.53,5865.59,2801.04,2801.04,2801.04,2801.04,2854.41,3067.43,39876.54


In [41]:
df_hogar.query("Subcategoria == 'Cine'")

,Fecha,Tipo,Categoria,Subcategoria,Beneficiario,Importe
919,2022-01-09,Gastos,Cultura,Cine,RENOIR,-26.0
